In [1]:
import numpy as np
import os
import time

import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

In [2]:
from dataclasses import dataclass

In [3]:
import math

In [4]:
vis = meshcat.Visualizer()


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [38]:
def get_vertices(direction):
    to_change = [i for i,_ in enumerate(direction) if direction[i] == 0]
    vertices = []

    for val1 in [-1, 1]:
        for val2 in [-1, 1]:
            vertex = list(direction)
            vertex[to_change[0]] = val1
            vertex[to_change[1]] = val2
            vertices.append(vertex)



    return vertices

def get_cube_mesh(faces2color: dict, size):
    
    faces = []
    colors = []
    vertices_cube = []
    face_index = 0
    for direction, color in faces2color.items():
        vertices_cube.extend(get_vertices(direction))
        faces.append([face_index + i for i in [0,1,2] ])
        faces.append([face_index + i for i in [3,2,1] ])
        face_index += 4
        colors.extend([color]*4)

    return g.TriangularMeshGeometry(np.array(vertices_cube)*size, np.array(faces), np.array(colors))

In [39]:
@dataclass
class Block:
    id: str
    transform: np.ndarray


In [40]:
def get_face_blocks(dimension, number, blocks, side):
    face_blocks = []
    for block in blocks:
        if math.isclose(block.transform[dimension, -1], number * side, abs_tol=1e-15):
            face_blocks.append(block)

    return face_blocks

In [41]:
def get_affine(rotation, translation):
    affine = np.ones((4,4))
    affine[0:3, 0:3] = rotation[0:3, 0:3]
    affine[0:3, -1] = translation[0:3, -1]
    return affine

In [42]:
def rotate_face(dimension, number, blocks, duration_sec, side):
    direction = np.zeros(3)
    direction[dimension] = 1
    X_WF = tf.translation_matrix(direction*side)
    X_FW = np.linalg.inv(X_WF)

    face_blocks = get_face_blocks(dimension, number, blocks, side)
    ang_fin = math.pi/2
    #frame_rate = 60
    steps = 50
    ts = np.linspace(0, duration_sec, steps)
    vel_ang = ang_fin/duration_sec
    for t in ts:
        for block in face_blocks:
            X_Fb = X_FW @ block.transform
            X_F_NF = tf.rotation_matrix(vel_ang * duration_sec/steps, direction)
            X_W_NF = X_WF @ X_F_NF
            block.transform = X_W_NF @ X_Fb
            
        for block in face_blocks:
            vis[f'cube/{block.id}'].set_transform(block.transform)
        time.sleep(duration_sec/steps)


In [43]:
red = [1, 0, 0]
green = [0, 1, 0]
blue = [0, 0, 1]
yellow = [1, 1, 0]
orange = [1, 0.5, 0]
white = [1, 1, 1]
black = [0, 0, 0]

def get_cube(x_coord, y_coord, z_coord, N, side):
    faces2color = {}
    for index in [0, 1, 2]:
        for dir in [1, -1]:
            direction = [0, 0, 0]
            direction[index] = dir
            faces2color[tuple(direction)] = black
    if x_coord == (N-1)/2:
        faces2color[(1, 0, 0)] = red
    if z_coord == (N-1)/2:
        faces2color[(0, 0, 1)] = yellow
    if y_coord == (N-1)/2:
        faces2color[(0, 1, 0)] = green

    if x_coord == -(N-1)/2:
        faces2color[(-1, 0, 0)] = orange
    if z_coord == -(N-1)/2:
        faces2color[(0, 0, -1)] = white
    if y_coord == -(N-1)/2:
        faces2color[(0, -1, 0)] = blue

    return get_cube_mesh(faces2color, side/2)
    


In [44]:
vis['cube'].delete()

In [45]:
coords2id = {}
blocks = []

N = 7
BLOCK_SIDE = 0.2

vis['cube'].set_object(g.Box([BLOCK_SIDE]*3))
vis['cube'].set_transform(tf.translation_matrix([0, 0, 0]))

offset = np.array([1, 1, 1]) * 0.05

if N%2 == 1:
    left = int(-(N-1)/2)
    right = int((N-1)/2 +1)
    steps = [i for i in range(left, right)]
for xi in steps:
    for yi in steps:
        for zi in steps:
            id = 'block' + str(xi) + str(yi) + str(zi)
            coords2id[(xi, yi, zi)] = id
            vis['cube'][id].set_object(get_cube(xi, yi, zi, N, BLOCK_SIDE), g.MeshPhongMaterial(vertexColors=True))
            transform = tf.translation_matrix(np.array([xi, yi, zi]) * BLOCK_SIDE)
            blocks.append(Block(id, transform))
            vis['cube'][id].set_transform(transform)

In [46]:
def scramble(N, n_times, blocks, duration_sec, side):
    for i in range(n_times):
        side_number = np.random.randint(-(N-1)/2, (N-1)/2 +1)
        dimension = np.random.randint(0, 3)
        rotate_face(dimension, side_number, blocks, duration_sec, side)


In [51]:
scramble(N, 10, blocks, 1, BLOCK_SIDE)

In [50]:
for i in range(20):
    vis['cube'].set_transform(tf.rotation_matrix(0.1*i, [0, 0, 1]))
    time.sleep(0.1)